In [41]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pickle
warnings.filterwarnings('ignore')

In [42]:
df_new = pd.read_pickle(r'C:\Users\nishi\Desktop\EECS 731 - Data Science\Project\df_new.pkl')

In [43]:
df_new.head()

,lpep_pickup_datetime,lpep_dropoff_datetime,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,...,total_amount,payment_type,trip_type,trip_duration,month,day,total_custom,weekend_Trip,hour,airport
0,2017-12-31 23:55:09,2017-12-31 23:59:14,1,17,61,1,1.1,5.5,0.5,0.5,...,6.80,2,1.0,4.083333,12,6,6.80,1,23,0
1,2017-12-31 23:30:36,2017-12-31 23:37:20,1,61,49,1,1.1,6.5,0.5,0.5,...,7.80,2,1.0,6.733333,12,6,7.80,1,23,0
2,2017-12-31 23:02:26,2017-12-31 23:19:37,1,228,188,1,4.1,15.5,0.5,0.5,...,20.15,1,1.0,17.183333,12,6,20.15,1,23,0
3,2017-12-31 23:12:24,2017-12-31 23:16:55,1,228,26,1,0.8,5.0,0.5,0.5,...,6.30,2,1.0,4.516667,12,6,6.30,1,23,0
4,2017-12-31 23:53:11,2017-12-31 23:55:40,1,225,225,1,0.5,4.0,0.5,0.5,...,5.30,2,1.0,2.483333,12,6,5.30,1,23,0


In [44]:
df_new.dtypes

lpep_pickup_datetime     datetime64[ns]
lpep_dropoff_datetime    datetime64[ns]
RatecodeID                        int64
PULocationID                      int64
DOLocationID                      int64
passenger_count                   int64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
payment_type                      int64
trip_type                       float64
trip_duration                   float64
month                             int64
day                               int64
total_custom                    float64
weekend_Trip                      int32
hour                              int64
airport                           int32
dtype: object

In [45]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression


X = df_new[['month','day','hour','PULocationID','DOLocationID','airport',
            'RatecodeID','passenger_count','payment_type','trip_type','weekend_Trip']]
y = df_new[['fare_amount']]

# Create and fit selector
selector = SelectKBest(f_regression, k=9)
selector.fit(X, y)
# Get idxs of columns to keep
idxs_selected = selector.get_support()
# Create new dataframe with only desired columns, or overwrite existing
features_dataframe_new = X.columns[idxs_selected]


In [46]:
features_dataframe_new

Index(['day', 'hour', 'PULocationID', 'DOLocationID', 'airport', 'RatecodeID',
       'passenger_count', 'payment_type', 'trip_type'],
      dtype='object')

In [47]:
# Creating a dataframes for model testing. Models cannot parse datetime columns, so have to drop them

df_1 = df_new[['day', 'hour', 'PULocationID', 'DOLocationID', 'airport', 'RatecodeID',
       'passenger_count', 'payment_type', 'trip_type']]


# Splitting the data into Training and Testing

In [48]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

features_1 = 
labels_1 = df_new[['fare_amount']]



In [49]:
# Split the data into training and testing sets

train_features_1, test_features_1, train_labels_1, test_labels_1 = train_test_split(features_1, labels_1, test_size = 0.10, random_state = 42)

In [50]:
print('Training Features Shape:', train_features_1.shape)
print('Training Labels Shape:', train_labels_1.shape)
print('Testing Features Shape:', test_features_1.shape)
print('Testing Labels Shape:', test_labels_1.shape)

Training Features Shape: (2024739, 9)
Training Labels Shape: (2024739, 1)
Testing Features Shape: (224972, 9)
Testing Labels Shape: (224972, 1)


# Using Random Forest for prediction

In [51]:
%%time
from sklearn.ensemble import RandomForestRegressor
rf1 = RandomForestRegressor()


rf1.fit(train_features_1, train_labels_1.values.ravel())

Wall time: 1min 8s


# Save the models to pickle file

# Read the models from pickle file

In [52]:
print('Accuracy of Random Forest on df_1 training set: {:.2f}'
     .format(rf1.score(train_features_1, train_labels_1)))
print('Accuracy of Random Forest on df_1 test set: {:.2f}'
     .format(rf1.score(test_features_1, test_labels_1)))


Accuracy of Random Forest on df_1 training set: 0.90
Accuracy of Random Forest on df_1 test set: 0.74


# Predicting from Input

In [53]:
prediction_1 = rf1.predict(test_features_1)

In [54]:
result_1 = test_features_1

In [55]:
result_1['actual_fare'] = test_labels_1[['fare_amount']]

result_1['predicted_fare'] = prediction_1


result_1['difference'] = abs(result_1['predicted_fare']-result_1['actual_fare'])


#### Saving the results to csv

In [56]:
result_1.head()

,day,hour,PULocationID,DOLocationID,airport,RatecodeID,passenger_count,payment_type,trip_type,actual_fare,predicted_fare,difference
1974340,1,22,74,74,0,1,1,2,1.0,5.5,5.318387,0.181613
884379,3,22,41,42,0,1,1,1,1.0,6.0,5.761894,0.238106
2030732,1,0,226,129,0,1,1,1,1.0,7.0,12.500000,5.500000
1990037,6,16,244,166,0,1,1,2,1.0,11.5,11.380357,0.119643
1866681,2,12,52,71,0,1,1,1,1.0,23.5,24.150000,0.650000


# Model Accuracy

Model_1 = {'month','day','hour','PULocationID','DOLocationID','fare_amount'}  
Model_2 = {'month','day','hour','PULocationID','DOLocationID','fare_amount','airport'}  
Model_3 = {'month','day','hour','PULocationID','DOLocationID','fare_amount','airport','trip_distance'}  
Model_4 = {'month','day','hour','PULocationID','DOLocationID','fare_amount','airport','trip_duration'}  
Model_5 = {'month','day','hour','PULocationID','DOLocationID','fare_amount','airport','trip_distance','trip_duration'}  

In [57]:
print("Model_1 Accuracy % : " +str((result_1[result_1['difference'] < 1].shape[0]/result_1.shape[0]) * 100)[:5])

Model_1 Accuracy % : 36.60
